# Stochastic Variables

The aim of the HydroRL model is to maximize the expected profits from generating electric power, given the uncertainty of inflow and energy price. In order for the model to learn, it interacts with the hydro system environment where it experiences different outcomes of the stochastic variables. The learned solution is thus highly dependent on how the stochastic variables are modeled, which we will outline in following.


## Background


In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from collections import Counter
from pathlib import Path
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

from core.timeindex import TimeIndexer, FromToTimeIndexer
from hps.exogenous.inflow_and_price import InflowPriceForecastData, InflowPriceSampler
from core.markov_chain import Noise

In [ ]:
plotly_template="ggplot2"
n_clusters = 4
sample_noise = Noise.StandardDev
n_drawn_samples = 10

In [ ]:
from hps_api_client.apiproxy  import ApiProxy
from hps_api_client.apiclient import ApiClient

In [ ]:
base_uri = "http://leviathan:5400/api/v1/"
api_client = ApiClient(base_uri)
client = ApiProxy(base_uri)

In [ ]:
projects = client.get_projects()
hydro_systems = client.get_hydro_systems()
forecasts = client.get_forecasts(hydro_systems[2])
forecasts

In [ ]:
forecast_uid = forecasts.to_pandas()["uid"][0]
forecast_uid

In [ ]:
def get_inflow_and_price(forecast_uid):
    forecast_scenarios = api_client.get_forecast_scenarios(forecast_uid)
    forecast_scenarios = forecast_scenarios["scenarios"][0] 
    forecast_scenario = forecast_scenarios[0]
    df = api_client.get_forecast_data(forecast_uid,forecast_scenario)

    dfi = pd.DataFrame(df["inflowSeries"][0], index=df["timeIndex"][0], columns = [forecast_scenario])
    dfi.index = pd.to_datetime(dfi.index)

    dfp = pd.DataFrame(df["priceSeries"][0], index=df["timeIndex"][0], columns = [forecast_scenario])
    dfp.index = pd.to_datetime(dfi.index)

    for forecast_scenario in forecast_scenarios[1:]:
        df = api_client.get_forecast_data(forecast_uid,forecast_scenario)
        dfi[forecast_scenario] = df["inflowSeries"][0]
        dfp[forecast_scenario] = df["priceSeries"][0]

    return dfi, dfp

In [ ]:
%%time
dfi, dfp = get_inflow_and_price(forecast_uid)

In order to provide an illustrative example, we will use some historical energy prices and inflows. We rearrange the data in order to obtain eight different scenarios that comprise our fictive forecast for what the inflow and energy price will be the next two years, given by the illustrations below.


In [ ]:
fig = dfi.plot(
    title="", labels=dict(index="Time", value="Inflow [m3/s]", variable="Historical year"), template=plotly_template)
fig.update_layout(title="")
fig.show()

In [ ]:
fig = dfp.plot(
    title="", labels=dict(index="Time", value="Energy price [m3/s]", variable="Historical year"), template=plotly_template)
fig.update_layout(title="")
fig.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

axs[0].plot(dfi.index, dfp.values);
axs[1].plot(dfi.index, dfi.values);

## Time index
We resample the forecast to a required time resolution. Say we want to train the model on a monthly basis, we get the following  inflow and energy prices after resampling.

In [ ]:
time_indexer = FromToTimeIndexer(dfi.index[0], dfi.index[-1], periods=24)
forecast_data = InflowPriceForecastData("test", dfi, dfp)
ip_sampler = InflowPriceSampler(forecast_data, time_indexer, n_clusters=n_clusters, sample_noise=sample_noise)

In [ ]:
fig = ip_sampler.df_i.plot(
    title="", labels=dict(index="Time", value="Inflow [m3/s]", variable="Historical year"), template=plotly_template)
fig.show()

In [ ]:
ip_sampler.df_p.plot(
    title="", labels=dict(index="Time", value="Energy price [m3/s]", variable="Historical year"), template=plotly_template)

## The Markov Chain
The HPS RL model requires a significant amount of training data. We generate the training data by sampling from a Markov chain that is generated from the resampled inflow and energy price data. The Markov chain is defined as 

\begin{equation}
    P(p_t = \chi^j_t \vert p_{t-1}=\chi^i_{t-1})=\rho_{ij}(t), \forall i, j \in M(t),
\end{equation}

where the transition probability, $\rho_{ij}(t)$, represents the probability of transitioning from node $i$ in time stage $t-1$  with uncertain data $\chi^i_{t-1}$ to node $j$ in time stage $t$. $p_t$ represents the realized data in stage $t$. $M(t)$ defines a set of nodes in time stage $t$.

The examples below illustrates a Markov chain with four nodes per time stage for the inflow and energy price data above. Note that the size of the markers illustrate the unconditioned weight of the nodes in a given time stage.


In [ ]:
# View clusters and samples
inf_clusters = np.array([cluster.cluster_centers_[:,1] for cluster in ip_sampler.forecast_generator.cls_])
price_clusters = np.array([cluster.cluster_centers_[:,0] for cluster in ip_sampler.forecast_generator.cls_])
counters = [Counter(cluster.labels_) for cluster in ip_sampler.forecast_generator.cls_]
n_steps, n_clusters = price_clusters.shape

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

axs[0].plot(ip_sampler.df_p.index, ip_sampler.df_p.values)
for i in range(n_clusters):
    for t in range(n_steps):
        axs[0].scatter(x=[time_indexer.index[t]], y=[price_clusters[t,i]], s=5*counters[t][i], color='k', zorder=1)

axs[1].plot(ip_sampler.df_i.index, ip_sampler.df_i.values)
for i in range(n_clusters):
    for t in range(n_steps):
        axs[1].scatter(x=[time_indexer.index[t]], y=[inf_clusters[t,i]], s=5*counters[t][i], color='k', zorder=1)

In [ ]:
price_transitions = []
time_transitions = []
for i, (prev, curr) in enumerate(zip(price_clusters[:-1], price_clusters[1:])):
    for c in curr:
        for p in prev:
            price_transitions.append([p, c])
            time_transitions.append([time_indexer.index[i], time_indexer.index[i+1]])
            
inflow_transitions = []
for prev, curr in zip(inf_clusters[:-1], inf_clusters[1:]):
    for c in curr:
        for p in prev:
            inflow_transitions.append([p, c])            

In [ ]:
std_dev = np.array(ip_sampler.forecast_generator.get_std_dev())
price_std = std_dev[:,0]
inflow_std = std_dev[:,1]

In [ ]:
data = []
for x, y in zip(time_transitions, price_transitions):
    data.append(
        go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(width=0.1, color='black'),
            showlegend=False
        )
    )

index = [i for i in time_indexer.index for _ in range(n_clusters)]
size = np.array([[i[1] for i in sorted(cnt.items())] for cnt in counters]).flatten()*4

data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_max[:,0],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Max value"
    )
)
data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_min[:,0],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Min value"
    )
)
data.append(
    go.Scatter(
        x= index,
        y=price_clusters.flatten(),
        mode='markers',
        marker=dict(size=size),
        name='Clusters'
    )
)
price_lower = price_clusters.min(axis=1) - price_std
price_upper = price_clusters.max(axis=1) + price_std
price_lower = price_lower[::-1]

data.insert(0,
    go.Scatter(
        x=time_indexer.index.tolist() + time_indexer.index[::-1].tolist(),
        y=np.concatenate((np.flip(price_lower, 0), np.flip(price_upper, 0)), axis=0),
        name="1 std. dev.",
        showlegend=True,
        fill="toself",
        fillcolor="grey",#'rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        opacity=0.1
    )
)

fig = go.Figure(data=data)
fig.update_layout(
    template=plotly_template, xaxis=dict(title="Time"), yaxis=dict(title="Energy price [EUR/MWh]"), showlegend=True)
fig.show()

In [ ]:
data = []
for x, y in zip(time_transitions, inflow_transitions):
    data.append(
        go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(width=0.1, color='black'),
            showlegend=False
        )
    )
    

inflow_lower = inf_clusters.min(axis=1) - inflow_std
inflow_upper = inf_clusters.max(axis=1) + inflow_std
inflow_lower = inflow_lower[::-1]

data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_max[:,1],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Max value"
    )
)
data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_min[:,1],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Min value"
    )
)


data.insert(0,
    go.Scatter(
        x=time_indexer.index.tolist() + time_indexer.index[::-1].tolist(),
        y=np.concatenate((np.flip(inflow_lower, 0), np.flip(inflow_upper, 0)), axis=0),
        name="1 std. dev.",
        showlegend=True,
        fill="toself",
        fillcolor="grey",#'rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        opacity=0.1
    )
)

data.append(go.Scatter(
    x= index,
    y=inf_clusters.flatten(),
    mode='markers',
    marker=dict(size=size),
    name="Clusters"
))

fig = go.Figure(data=data)
fig.update_layout(
    template=plotly_template, xaxis=dict(title="Time"), yaxis=dict(title="Inflow [m3/s]"), showlegend=True)
fig.show()

The shaded area denoted by 1 std. dev represents the one standard deviation between the nodes in the Markov chain and the underlying data. Thus giving an indication on how well the Markov chain fits the underlying data. There are three different configuration one can choose when sampling from the Markov chain. They are defined by setting the noise value to either "Off", "White" or "StandardDev" in the *RunSettings* object of the Web API. Their behavior is defined as

* "Off"
    * The sampled values from the Markov chain is given by the value of the nodes.
* "White"
    * White noise ($N(0,1)$) is added to the sampled values from the Markov chain.
* "StandardDev"
    * Noise defined by $N(0,\sigma^2)$ is added to the sampled values from the Markov chain.
    
Sine adding noise to the Markov chain values might be non-positive the values are clipped by some bound. The bounds are given such that the inflow has to be non-negative and the expected inflow is not shifted. The energy price is clipped such that only non-negative values are observed.

Below is an illustration with 10 sampled values with the noise parameter set to "StandardDev".

In [ ]:
sampled_episodes = []
for i in range(n_drawn_samples):
    episode, name = ip_sampler.sample_episode()
    sampled_episodes.append(episode)

In [ ]:
data = []

for x, y in zip(time_transitions, inflow_transitions):
    data.append(
        go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(width=0.1, color='black'),
            showlegend=False
        )
    )
    

inflow_lower = inf_clusters.min(axis=1) - inflow_std
inflow_upper = inf_clusters.max(axis=1) + inflow_std
inflow_lower = inflow_lower[::-1]

data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_max[:,1],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Max value"
    )
)
data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_min[:,1],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Min value"
    )
)


data.insert(0,
    go.Scatter(
        x=time_indexer.index.tolist() + time_indexer.index[::-1].tolist(),
        y=np.concatenate((np.flip(inflow_lower, 0), np.flip(inflow_upper, 0)), axis=0),
        name="1 std. dev.",
        showlegend=True,
        fill="toself",
        fillcolor="grey",#'rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        opacity=0.1
    )
)

data.append(go.Scatter(
    x= index,
    y=inf_clusters.flatten(),
    mode='markers',
    marker=dict(size=size),
    name="Clusters"
))

if True:
    for i, episode in enumerate(sampled_episodes):
        data.append(go.Scatter(
            x= time_indexer.index,
            y=episode[:,1],
            name=f"Sample {i}"
        ))

# data+=orig_data
fig = go.Figure(data=data)
fig.update_layout(
    template=plotly_template, xaxis=dict(title="Time"), yaxis=dict(title="Inflow [m3/s]"), showlegend=True)
fig.show()

In [ ]:
data = []
a=2

for x, y in zip(time_transitions, price_transitions):
    data.append(
        go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(width=0.1, color='black'),
            showlegend=False
        )
    )

index = [i for i in time_indexer.index for _ in range(n_clusters)]
size = np.array([[i[1] for i in sorted(cnt.items())] for cnt in counters]).flatten()*4

data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_max[:,0],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Max value"
    )
)
data.append(
    go.Scatter(
        x=time_indexer.index,
        y=ip_sampler.sample_min[:,0],
        mode='lines',
        line=dict(width=1, color='black'),
        name="Min value"
    )
)
data.append(
    go.Scatter(
        x= index,
        y=price_clusters.flatten(),
        mode='markers',
        marker=dict(size=size),
        name='Clusters'
    )
)
price_lower = price_clusters.min(axis=1) - price_std
price_upper = price_clusters.max(axis=1) + price_std
price_lower = price_lower[::-1]

data.insert(0,
    go.Scatter(
        x=time_indexer.index.tolist() + time_indexer.index[::-1].tolist(),
        y=np.concatenate((np.flip(price_lower, 0), np.flip(price_upper, 0)), axis=0),
        name="1 std. dev.",
        showlegend=True,
        fill="toself",
        fillcolor="grey",#'rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        opacity=0.1
    )
)

if True:
    for i, episode in enumerate(sampled_episodes):
        data.append(go.Scatter(
            x= time_indexer.index,
            y=episode[:,0],
            name=f"Sample {i}"
        ))

fig = go.FigureWidget(data=data)
fig.update_layout(
    template=plotly_template, xaxis=dict(title="Time"), yaxis=dict(title="Energy price [EUR/MWh]"), showlegend=True)
fig.show(renderer="notebook")